In [ ]:
# Set the Datalake Access Key configuration
spark.conf.set(
    "fs.azure.account.key.cryptoanalyticslake.dfs.core.windows.net",
    dbutils.secrets.get(scope="key-vault-secret-scope",key="cryptoanalyticslake-access-key"))

In [ ]:
# Use secrets DBUtil to get Snowflake credentials.
user = dbutils.secrets.get(scope="key-vault-secret-scope",key="snowflake-username")
password = dbutils.secrets.get(scope="key-vault-secret-scope",key="snowflake-password")
url = dbutils.secrets.get(scope="key-vault-secret-scope",key="snowflake-url")

# snowflake connection options
options = {
  "sfUrl": url,
  "sfUser": user,
  "sfPassword": password,
  "sfDatabase": "SHARED_CRYPTO_QUOTES",
  "sfSchema": "PUBLIC",
  "sfWarehouse": "READER_WH"
}

In [ ]:
# spark.sql("CREATE SCHEMA IF NOT EXISTS gold")

In [ ]:
# sql_table_create = "CREATE TABLE IF NOT EXISTS gold.quotes \
#                       (Symbol STRING NOT NULL, Price DOUBLE NOT NULL, PriceTimeStamp TIMESTAMP NOT NULL, PriceDate DATE NOT NULL) \
#                     USING DELTA \
#                     PARTITIONED BY (PriceDate) \
#                     LOCATION 'abfss://crypto-gold@cryptoanalyticslake.dfs.core.windows.net/quotes-delta-table'"

# spark.sql(sql_table_create)

In [ ]:
# sql_copy = f"COPY INTO gold.quotes \
#              FROM (SELECT _c0, _c1, _c2, _c3 \
#                 FROM 'abfss://crypto-gold@cryptoanalyticslake.dfs.core.windows.net/quotes-by-day-manual-partition-single-file/{year}/{month:0>2d}/{day:0>2d}') \
#              FILEFORMAT = CSV "

# spark.sql(sql_copy)

In [ ]:
# Set Day Month Year
from datetime import datetime

today = datetime.utcnow()
year = today.year
month = today.month
day = today.day

In [ ]:
singlecsvfolderpath = f"abfss://crypto-gold@cryptoanalyticslake.dfs.core.windows.net/quotes-by-day-manual-partition-single-file/{year}/{month:0>2d}/{day:0>2d}"

df = spark.read.format("csv").option("header", "true").load(singlecsvfolderpath)

df.write \
    .format("snowflake") \
    .options(**options) \
    .option("dbtable", "CRYPTO_QUOTES") \
    .mode("append") \
    .save()

